## IPL Re-predictions

In [1]:
import csv
import math
import time
import itertools
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
from scipy.stats import linregress

import datetime as dt 
from dateutil.parser import parse
from datetime import datetime, timedelta
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df_ads = pd.read_csv('gs://aes-analytics-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_2020_11162020.csv')
df_ads = df_ads[df_ads.TTR<5000]
df_ads.reset_index(drop=True, inplace=True)
print(df_ads.shape)
df_ads.head()

(8429, 157)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Active_no_of_trucks_per_day
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,False,147.00,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00,0,0,0,0,0,0,0,0,0,0,0,Wednesday,False,1,1.00,0.18,1.00,MILL,0.00,27.00,100.00,25.50,27.90,29.80,26.20,33.40,42.80,31.80,38.60,46.70,1007.00,1010.70,1013.50,0.00,118.60,499.20,2846.90,43.20,67.00,89.00,981.60,984.80,987.40,0.00,3.00,3.30,3.50,31.80,38.60,46.70,26.20,33.40,42.80,220.00,216.00,218.00,5.00,7.20,10.40,12.50,18.00,28.60,13.00,17.90,27.70,30.50,34.70,39.30,14.90,5.40,5.80,100.00,45.80,2.06,1.01,nan
1,2002646057,2001465296.00,281-B/193,BLUE SKY,2020-01-01 03:10:16,2020-01-01 11:46:19,3607.00,FUSE,360.00,3,2.00,0.00,2020-01-01 07:45:00,2.00,False,516.05,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,39.88,-86.02,ZONE3,LOCATION CREATED BY OPENING DEVICE: 281-B/193 ...,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,2,1.00,0.00,1.00,ARL.,0.00,28.00

In [3]:
def ceil_dt(dt, delta):
    return dt + (datetime.min - dt) % delta

def convert_timestamp(func):
    '''
    Decorator to make p1, p2 value a timestamp value
    To perform easy timestamp operations
    '''
    def inner(p1, p2):
        p1 = datetime.strptime(p1, "%Y-%m-%d %H:%M:%S")
        p2 = datetime.strptime(p2, "%Y-%m-%d %H:%M:%S")

        return func(p1, p2)
    return inner

In [4]:
@convert_timestamp
def add_30_mins(start_timestamp, end_timestamp):
    new_timestamp_list = []
    new_timestamp_list.append((ceil_dt(start_timestamp, timedelta(minutes=30))).strftime("%Y-%m-%d %H:%M:%S"))
    new_timestamp = (ceil_dt(start_timestamp, timedelta(minutes=30)))
    while new_timestamp < end_timestamp:
        new_timestamp = new_timestamp + timedelta(minutes=60)
        if new_timestamp > end_timestamp:
            break
        else :
            new_timestamp_list.append(new_timestamp.strftime("%Y-%m-%d %H:%M:%S"))
    
    return new_timestamp_list

In [5]:
def create_time_groups(df):
    time_group_list = []
    for i in range(len(df)):
        time_intervals = add_30_mins(df['CREATION_DATETIME'][i], df['ENERGIZED_DATETIME'][i])
        time_group_list.append(time_intervals)
    
    return time_group_list

time_intervals_group = create_time_groups(df_ads)

In [6]:
list_len = []
for i in range(len(time_intervals_group)):
    list_new = len(time_intervals_group[i])
    list_len.append(list_new)

In [7]:
len(list_len)

8429

In [8]:
final_timestamp = list(itertools.chain(*time_intervals_group))

In [9]:
print(len(list_len))
print(len(final_timestamp))

8429
34162


In [10]:
test = df_ads.copy(deep=True)
test['Times'] = list_len

In [11]:
test_v1 = test.loc[test.index.repeat(test.Times)].reset_index(drop=True)
test_v1['Time_Bucket'] = final_timestamp

In [12]:
test_v1['CREATION_DATETIME'] = pd.to_datetime(test_v1['CREATION_DATETIME'], errors='coerce')
test_v1['ENERGIZED_DATETIME'] = pd.to_datetime(test_v1['ENERGIZED_DATETIME'], errors='coerce')
test_v1['Time_Bucket'] = pd.to_datetime(test_v1['Time_Bucket'], errors='coerce')

test_v1['Time_Since_Outage'] = (test_v1.Time_Bucket - test_v1.CREATION_DATETIME).dt.total_seconds().div(60).round(4)

test_v1['TR'] = (test_v1.ENERGIZED_DATETIME - test_v1.Time_Bucket).dt.total_seconds().div(60).round(4)

In [13]:
test_v1.sort_values(by=['CREATION_DATETIME', 'Time_Bucket'], inplace=True)
test_v1.reset_index(drop=True, inplace=True)
print(test_v1.shape)

(34162, 161)


In [14]:
test_v1['REP_OUTAGE_ID'] = test_v1['OUTAGE_ID'].astype(str) + test_v1['Time_Bucket'].astype(str).str[8:16]
outage_Count_df = test_v1[['OUTAGE_ID','CREATION_DATETIME']].drop_duplicates()

In [15]:
print(list(test_v1.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [16]:
test_v1.drop(columns= ['Outages_in_last_1hr', 'Outages_in_last_2hr', 'Outages_in_last_3hr', 'Outages_in_last_4hr',
                       'Outages_in_last_5hr', 'Outages_in_last_6hr', 'Outages_in_last_7hr', 'Outages_in_last_8hr',
                       'Outages_in_last_9hr', 'Outages_in_last_10hr'], inplace=True)

In [17]:
test_v1.reset_index(drop=True, inplace=True)
print(test_v1.shape)

(34162, 152)


In [18]:
# importing the multiprocessing module 


from multiprocessing import Pool
import tqdm
import time

def count_outage(group):
    group = group.reset_index(drop = True)
    df_temp = outage_Count_df[['OUTAGE_ID','CREATION_DATETIME']]
    df_temp['minutes'] = (group['Time_Bucket'][0] - outage_Count_df['CREATION_DATETIME']).dt.total_seconds().div(60)
    df_temp = df_temp[df_temp.minutes > 0]
    df_temp = df_temp[df_temp.OUTAGE_ID != group.OUTAGE_ID[0]]
    group['Outages_in_last_1hr'] = len(df_temp[df_temp.minutes <= 60])
    group['Outages_in_last_2hr'] = len(df_temp[df_temp.minutes <= 120])
    group['Outages_in_last_3hr'] = len(df_temp[df_temp.minutes <= 180])
    group['Outages_in_last_4hr'] = len(df_temp[df_temp.minutes <= 240])
    group['Outages_in_last_5hr'] = len(df_temp[df_temp.minutes <= 300])
    group['Outages_in_last_6hr'] = len(df_temp[df_temp.minutes <= 360])
    group['Outages_in_last_7hr'] = len(df_temp[df_temp.minutes <= 420])
    group['Outages_in_last_8hr'] = len(df_temp[df_temp.minutes <= 480])
    group['Outages_in_last_9hr'] = len(df_temp[df_temp.minutes <= 540])
    group['Outages_in_last_10hr'] = len(df_temp[df_temp.minutes <= 600])
    return group

def grouping_fn(df):
    outage = df.groupby(['REP_OUTAGE_ID'], as_index=False).apply(count_outage)
    return outage
    
if __name__ == '__main__':
    starttime = time.time()
    with Pool(50) as p:
        outage_n_hours =  list(tqdm.tqdm(p.imap(grouping_fn, [test_v1[:10000],test_v1[10000:20000],
                                                              test_v1[20000:30000], test_v1[30000:40000], 
                                                              test_v1[40000:50000], test_v1[50000:60000],
                                                              test_v1[60000:80000], test_v1[80000:85000],
                                                              test_v1[85000:90000],
                                                              test_v1[90000:100000], test_v1[100000:110000],
                                                              test_v1[110000:130000], test_v1[130000:140000],
                                                              test_v1[140000:150000], test_v1[150000:160000], 
                                                              test_v1[160000:170000], test_v1[170000:175000],
                                                              test_v1[175000:180000],
                                                              test_v1[180000:190000], test_v1[190000:200000],
                                                              test_v1[200000:220000], test_v1[220000:230000],
                                                              test_v1[230000:240000], test_v1[2400000:250000],
                                                              test_v1[250000:260000], test_v1[260000:280000],
                                                              test_v1[280000:300000], test_v1[300000:320000],
                                                              test_v1[320000:340000], test_v1[340000:350000],
                                                              test_v1[350000:355000], test_v1[355000:360000],
                                                              test_v1[360000:370000], test_v1[370000:380000],
                                                              test_v1[380000:390000], test_v1[390000:400000],
                                                              test_v1[400000:410000], test_v1[410000:420000],
                                                              test_v1[420000:430000], test_v1[430000:440000],
                                                              test_v1[440000:450000], test_v1[450000:460000],
                                                              test_v1[460000:490000], test_v1[490000:500000],
                                                              test_v1[500000:510000], test_v1[510000:520000],
                                                              test_v1[520000:530000], test_v1[530000:540000],
                                                              test_v1[540000:]]), total=100))

 49%|████▉     | 49/100 [03:39<03:48,  4.49s/it]  


In [19]:
ads_df = pd.concat(outage_n_hours)

In [20]:
print(list(ads_df.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [21]:
ads_df.shape

(34162, 162)

In [22]:
def filter_low(group):
    group.reset_index(inplace = True)
    if(group.Time_Since_Outage.max() <= 60):
        return group.OUTAGE_ID[0]
    else:
        return 1
outage_id_rem = ads_df.groupby('OUTAGE_ID').apply(filter_low).reset_index(drop = True)

In [23]:
ads_df = ads_df[~ads_df.OUTAGE_ID.isin(outage_id_rem)]
ads_df.reset_index(drop=True, inplace=True)

In [24]:
ads_df[['OUTAGE_ID', 'CREATION_DATETIME', 'Time_Bucket', 'ENERGIZED_DATETIME', 'Time_Since_Outage', 'TR']].head(20)

,OUTAGE_ID,CREATION_DATETIME,Time_Bucket,ENERGIZED_DATETIME,Time_Since_Outage,TR
0,2002645939,2020-01-01 00:03:00,2020-01-01 00:30:00,2020-01-01 02:30:00,27.00,120.00
1,2002645939,2020-01-01 00:03:00,2020-01-01 01:30:00,2020-01-01 02:30:00,87.00,60.00
2,2002645939,2020-01-01 00:03:00,2020-01-01 02:30:00,2020-01-01 02:30:00,147.00,0.00
3,2002645982,2020-01-01 03:23:08,2020-01-01 03:30:00,2020-01-01 05:09:12,6.87,99.20
4,2002645982,2020-01-01 03:23:08,2020-01-01 04:30:00,2020-01-01 05:09:12,66.87,39.20
5,2002645992,2020-01-01 03:10:16,2020-01-01 03:30:00,2020-01-01 05:23:41,19.73,113.68
6,2002645992,2020-01-01 03:10:16,2020-01-01 04:30:00,2020-01-01 05:23:41,79.73,53.68
7,2002646017,2020-01-01 03:23:07,2020-01-01 03:30:00,2020-01-01 04:45:07,6.88,75.12
8,2002646017,2020-01-01 03:23:07,2020-01-01 04:30:00,2020-01-01 04:45:07,66.88,15.12
9,2002646018,2020-01-01 03:23:07,2020-01-01 03:30:00,2020-01-01 04:44:22,6.88,74.37


In [28]:
def feature_add(group):
    group = group.reset_index(drop = True)
    x = [group.Outages_in_last_1hr[0],group.Outages_in_last_2hr[0],group.Outages_in_last_3hr[0],group.Outages_in_last_4hr[0],group.Outages_in_last_5hr[0]
        ,group.Outages_in_last_6hr[0],group.Outages_in_last_7hr[0],group.Outages_in_last_8hr[0],group.Outages_in_last_9hr[0],group.Outages_in_last_10hr[0]]
    
    y = [1,2,3,4,5,6,7,8,9,10]
    
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    group['Slope_outages'] = slope
    
    ROC = [abs(group.Outages_in_last_1hr[0] - group.Outages_in_last_2hr[0]),abs(group.Outages_in_last_2hr[0] - group.Outages_in_last_3hr[0]),
           abs(group.Outages_in_last_3hr[0] - group.Outages_in_last_4hr[0]),abs(group.Outages_in_last_4hr[0] - group.Outages_in_last_5hr[0]),
           abs(group.Outages_in_last_5hr[0] - group.Outages_in_last_6hr[0]),abs(group.Outages_in_last_6hr[0] - group.Outages_in_last_7hr[0]),
           abs(group.Outages_in_last_7hr[0] - group.Outages_in_last_8hr[0]),abs(group.Outages_in_last_8hr[0] - group.Outages_in_last_9hr[0]),
           abs(group.Outages_in_last_9hr[0] - group.Outages_in_last_10hr[0])]
    y = [1,2,3,4,5,6,7,8,9]
    slope, intercept, r_value, p_value, std_err = linregress(ROC, y)
    group['Slope_ROC_outages'] = slope
    
    group['max_ROC_outages'] = max(ROC)
    group['max_index_ROC'] = 1+ROC.index(max(ROC))    
    group['weight_ROC'] = group['max_ROC_outages'] * group['max_index_ROC']*0.1
    return group

In [29]:
ads_df = ads_df.groupby(['OUTAGE_ID'], as_index = False).apply(feature_add).reset_index(drop = True)

In [30]:
print(list(ads_df.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [31]:
ads_df.dropna(subset=['Slope_ROC_outages', 'Slope_outages'],inplace=True)
ads_df.drop(['Times', 'Time_Bucket', 'TTR', 'Active_no_of_trucks_per_day', 'REP_OUTAGE_ID', 'EVENT', 'LAT', 'LONG', 'Marker_Location',
              'RANK_SUBSEQUENT_OUTAGES', 'Live_outage_group', 'Dis_From_Live_Centriod_div_Cust_qty'], axis=1, inplace=True)
ads_df.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Hour_Sin,Hour_Cos,LIVE_OUTAGE,Weekday,Weekend_flag,Priority_Customer_Qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Time_Since_Outage,TR,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Slope_outages,Slope_ROC_outages,max_ROC_outages,max_index_ROC,weight_ROC
3,2002645982,2001465252.00,318--/116,2020-01-01 03:23:08,2020-01-01 05:09:12,3607.00,1TBOH,360.00,3,4.00,0.00,2020-01-01 07:00:00,4.00,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,ZONE3,NaN,37,2,17,8,5,0.71,0.71,13,Wednesday,False,16.00,21.00,ARL.,0.00,27.00,100.00,26.70,28.30,30.70,25.90,32.30,41.50,31.90,38.30,46.00,1006.90,1010.50,1013.20,0.00,117.40,497.40,2817.30,47.00,69.00,89.70,979.40,982.50,985.00,0.00,3.10,3.30,3.70,31.90,38.30,46.00,25.90,32.30,41.50,221.00,218.00,220.00,5.30,8.50,12.20,12.40,18.20,28.00,13.30,17.90,26.80,30.50,34.60,39.10,14.10,6.90,5.60,100.00,42.70,1.91,1.01,6.87,99.20,13,13,13,14,14,14,14,14,14,14,5.00,-2.25,1,3,0.30
4,2002645982,2001465252.00,318--/116,2020-01-01 03:23:08,2020-01-01 05:09:12,3607.00,1TBOH,360.00,3,4.00,0.00,2020-01-01 07:00:00,4.00,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,ZONE3,NaN,37,2,17,8,5,0.71,0.71,13,Wednesday,False,16.00,21.00,ARL.,0.00,27.00,100.00,26.70,28.30,30.70,25.90,32.30,41.50,31.90,38.30,46.00,1006.90,1010.50,1013.20,0.00,117.40,497.40,2817.30,47.00,69.00,89.70,979.40,982.50,985.00,0.00,3.10,3.

In [32]:
ads_df.to_csv('gs://aes-analytics-0002-curated/Outage_Restoration/Repredictions/Master_Dataset/OMS_IPL_Repredictions_2020.csv')